In [17]:
import pandas as pd
import osmnx as ox
import pandas as pd
from geopy.distance import geodesic

In [4]:
os.chdir(r"C:\Users\dsala\Downloads")

In [7]:
train=pd.read_csv("train_barrios.csv")
test=pd.read_csv("test_barrios.csv")

In [ ]:



ox.config(use_cache=True, log_console=True)

# Descargar datos de OpenStreetMap para Bogotá

bogota_boundary = ox.geocoder.geocode_to_gdf("Bogotá, Colombia")

# Obtener parques como puntos de interés
parks = ox.geometries_from_polygon(
    bogota_boundary.geometry.iloc[0], tags={"leisure": "park"}
)

# Obtener estaciones de transporte público como puntos de interés
transport_stations = ox.geometries_from_polygon(
    bogota_boundary.geometry.iloc[0], tags={"public_transport": True}
)

Hospitales=ox.geometries_from_polygon(
    bogota_boundary.geometry.iloc[0], tags={"amenity": "hospital"}
)


# Extraer las coordenadas de los POIs
def extract_poi_coordinates(gdf, name_column=None):
    """
    Extrae coordenadas (lat, lon) de un GeoDataFrame.
    """
    pois = gdf[['geometry']].copy()
    pois['lat'] = pois.geometry.centroid.y
    pois['lon'] = pois.geometry.centroid.x
    if name_column and name_column in gdf.columns:
        pois['name'] = gdf[name_column]
    else:
        pois['name'] = 'Unknown'
    return pois[['name', 'lat', 'lon']]

# Crear DataFrames con POIs
park_pois = extract_poi_coordinates(parks, name_column="name")
transport_pois = extract_poi_coordinates(transport_stations, name_column="name")
hospital_pois = extract_poi_coordinates(Hospitales, name_column="name")



# Guardar los POIs en archivos CSV
park_pois.to_csv("parks_pois.csv", index=False)
transport_pois.to_csv("transport_stations_pois.csv", index=False)
hospital_pois.to_csv("hospital_pois.csv", index=False)

print("POIs guardados como 'parks_pois.csv' y 'transport_stations_pois.csv'")


C:\Users\dsala\AppData\Local\Temp\ipykernel_3984\3720497210.py:2: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.config(use_cache=True, log_console=True)
C:\Users\dsala\AppData\Local\Temp\ipykernel_3984\3720497210.py:9: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  parks = ox.geometries_from_polygon(
C:\Users\dsala\AppData\Local\Temp\ipykernel_3984\3720497210.py:14: FutureWarning: The `geometries` module and `geometries_from_X` fu

POIs guardados como 'parks_pois.csv' y 'transport_stations_pois.csv'


C:\Users\dsala\AppData\Local\Temp\ipykernel_3984\3720497210.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois['lat'] = pois.geometry.centroid.y
C:\Users\dsala\AppData\Local\Temp\ipykernel_3984\3720497210.py:30: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois['lon'] = pois.geometry.centroid.x
C:\Users\dsala\AppData\Local\Temp\ipykernel_3984\3720497210.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois['lat'] = pois.geometry.centroid.y
C:\Users\dsala\AppData\Local\Temp\ipykernel_3984\3720497210.py:30: UserWarning: Geometry is in a geographic CRS. Results 

In [19]:

from scipy.spatial import cKDTree
import numpy as np


# Cargar datos de POIs
hospital_pois = pd.read_csv('hospital_pois.csv')
park_pois = pd.read_csv('parks_pois.csv')
transport_pois = pd.read_csv('transport_stations_pois.csv')

# Función para calcular las distancias mínimas usando cKDTree
def calculate_nearest_poi_fast(properties, pois):
    """
    Calcula la distancia mínima desde cada propiedad a un conjunto de POIs utilizando cKDTree.
    """
    # Crear un árbol KD con los POIs
    tree = cKDTree(pois)
    # Calcular distancias mínimas para cada propiedad
    distances, _ = tree.query(properties, k=1)  # k=1 para la distancia más cercana
    return distances

# Convertir coordenadas en matrices NumPy para optimización
def get_coordinates(df):
    return np.array(df[['lat', 'lon']])

# Coordenadas de las propiedades
train_coords = get_coordinates(train)
test_coords = get_coordinates(test)

# Coordenadas de los POIs
hospital_coords = get_coordinates(hospital_pois)
park_coords = get_coordinates(park_pois)
transport_coords = get_coordinates(transport_pois)

# Calcular distancias mínimas y añadirlas a las bases
train['distance_to_nearest_hospital'] = calculate_nearest_poi_fast(train_coords, hospital_coords)
train['distance_to_nearest_park'] = calculate_nearest_poi_fast(train_coords, park_coords)
train['distance_to_nearest_transport_station'] = calculate_nearest_poi_fast(train_coords, transport_coords)

test['distance_to_nearest_hospital'] = calculate_nearest_poi_fast(test_coords, hospital_coords)
test['distance_to_nearest_park'] = calculate_nearest_poi_fast(test_coords, park_coords)
test['distance_to_nearest_transport_station'] = calculate_nearest_poi_fast(test_coords, transport_coords)

# Guardar los resultados en archivos
train.to_csv('train_with_distances.csv', index=False)
test.to_csv('test_with_distances.csv', index=False)

print("Cálculo completado con optimización. Archivos guardados como 'train_with_distances.csv' y 'test_with_distances.csv'")





Cálculo completado con optimización. Archivos guardados como 'train_with_distances.csv' y 'test_with_distances.csv'


In [25]:


# Coordenadas del centro financiero de Bogotá (Calle 72 con Carrera Séptima)
financial_center_coords = (4.6581, -74.0578)

# Función para calcular la distancia al centro financiero
def calculate_distance_to_financial_center(lat, lon):
    property_coords = (lat, lon)
    return geodesic(property_coords, financial_center_coords).meters



# Añadir la distancia al centro financiero
train['distance_to_financial_center'] = train.apply(
    lambda row: calculate_distance_to_financial_center(row['lat'], row['lon']),
    axis=1
)
test['distance_to_financial_center'] = test.apply(
    lambda row: calculate_distance_to_financial_center(row['lat'], row['lon']),
    axis=1
)






In [27]:
train.to_csv('train_PS3.csv', index=False)
test.to_csv('test_PS3.csv', index=False)

In [28]:
train.columns

Index(['property_id', 'city', 'price', 'month', 'year', 'surface_total',
       'surface_covered', 'rooms', 'bedrooms', 'bathrooms', 'property_type',
       'operation_type', 'lat', 'lon', 'title', 'description', 'barrio',
       'cerca_virrey', 'num_parqueaderos', 'num_banos',
       'distance_to_nearest_hospital', 'distance_to_nearest_park',
       'distance_to_nearest_transport_station',
       'distance_to_financial_center'],
      dtype='object')